In [11]:
import pandas as pd
import numpy as np

from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt


## Read in May 2023 OEWS Data

BLS OEWS publishes employment estimates for industies and SOC Codes on https://www.bls.gov/oes/tables.htm. Use the 'all data' for employment estimates for the U.S. economy.

In [2]:
filepath = r'C:\Users\maurice\OneDrive\Government Data'
df = pd.read_excel(filepath + '\\all_data_M_2023.xlsx')

This analysis is only interested in the most detailed SOC Codes in the U.S. economy.

In [3]:
filtered = df[(df['NAICS']!='000000')&(df['NAICS']!='000001')&(df['O_GROUP']=='detailed')]

In [4]:
keep_columns = ['NAICS','OCC_CODE','TOT_EMP']
unique_soc_naics = filtered[['NAICS','OCC_CODE']].drop_duplicates()
full_list = pd.merge(filtered, unique_soc_naics, on=['NAICS','OCC_CODE'], how='inner')[keep_columns]

## Non-Standard NAICS

Non-standard NAICS codes. The following are non-standard NAICS codes.

In [5]:
key = ['NAICS','OCC_CODE']
non_standard = full_list[full_list['NAICS'].apply(lambda x: str(x).isalnum() and any(c.isalpha() for c in str(x)) and any(c.isdigit() for c in str(x)))]
standard_list = full_list[~full_list[key].apply(tuple, axis=1).isin(non_standard[key].apply(tuple, axis=1))]

## Expand NAICS Code (if hyphenated)

If there is a hyphen in the NAICS code, we want to create a new row so that it is 31, 32, 33.

In [6]:
def expand_naics(df):
   # Create copy of dataframe
   expanded_df = df.copy()
   
   # Find rows with hyphens
   mask = expanded_df['NAICS'].str.contains('-', na=False)
   hyphen_rows = expanded_df[mask].copy()
   
   # Drop original hyphen rows
   expanded_df = expanded_df[~mask]
   
   # Expand hyphenated rows
   new_rows = []
   for idx, row in hyphen_rows.iterrows():
       start, end = map(int, row['NAICS'].split('-'))
       for naics in range(start, end + 1):
           new_row = row.copy()
           new_row['NAICS'] = str(naics)
           new_rows.append(new_row)
   
   # Combine original and expanded rows
   expanded_df = pd.concat([expanded_df, pd.DataFrame(new_rows)], ignore_index=True)
   
   return expanded_df

standard_list = expand_naics(standard_list)

## Process Government NAICS

**Non-standard.** NAICS code 99 is not a standard NAICS for government workers. It should be NAICS 92.

In [7]:
government_naics = ['999000', '999001', '999100', '999101', '999200','999201', '999300', '999301','99']
government = standard_list[standard_list['NAICS'].isin(government_naics)]
standard_list = standard_list[~standard_list['NAICS'].isin(government_naics)]
non_standard = pd.concat([government, non_standard])
non_standard

,NAICS,OCC_CODE,TOT_EMP
6545,99,11-1011,25550
6546,99,11-1021,135720
6547,99,11-1031,32460
6548,99,11-2011,100
6549,99,11-2021,1820
...,...,...,...
82149,5320A1,53-7061,1710
82150,5320A1,53-7062,28440
82151,5320A1,53-7065,3570
82152,5320A1,53-7072,560


**Standard**. The standard NAICS/SOC Codes for government workers. It should exclude the aggregated NAICS and NAICS 99 because NAICS is not standard representing government (NAICS 92)

In [8]:
standard_list

,NAICS,OCC_CODE,TOT_EMP
0,11,11-1011,130
1,11,11-1021,6020
2,11,11-2021,60
3,11,11-2022,350
4,11,11-3012,260
...,...,...,...
81260,49,53-7081,440
81261,48,53-7121,6900
81262,49,53-7121,6900
81263,48,53-7199,4800


## Cluster Lists

Assign a clustering algorithm based on TOT_EMP to identify thresholds based on TOT_EMP in a NAICS / OCC_CODE pair.

+  **Standard Lists.** The clustering algorithm based on the list of standard NAICS / OCC_Codes.

In [17]:

def gmm_cluster(df):

   # Replace '**' with NaN and drop those rows
   df['TOT_EMP'] = df['TOT_EMP'].replace('**', np.nan)
   df = df.dropna(subset=['TOT_EMP'])

   # Convert to numeric
   df['TOT_EMP'] = pd.to_numeric(df['TOT_EMP'])
   # Prepare data
   X = df[['TOT_EMP']].values
   scaler = StandardScaler()
   X_scaled = scaler.fit_transform(X)

   # Fit GMM
   n_components = 3  # Adjust number of clusters as needed
   gmm = GaussianMixture(n_components=n_components, random_state=42)
   df['GMM_CLUSTER'] = gmm.fit_predict(X_scaled)
   sorted_df = df.sort_values(by='TOT_EMP',ascending=False)

   return sorted_df

gmm_cluster(standard_list)

C:\Users\maurice\AppData\Local\Temp\ipykernel_31280\3600139752.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TOT_EMP'] = pd.to_numeric(df['TOT_EMP'])
C:\Users\maurice\AppData\Local\Temp\ipykernel_31280\3600139752.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GMM_CLUSTER'] = gmm.fit_predict(X_scaled)


,NAICS,OCC_CODE,TOT_EMP,GMM_CLUSTER
5106,62,31-1120,3575980.0,1
80018,45,41-2031,3319640.0,1
80017,44,41-2031,3319640.0,1
5836,72,35-3023,3220260.0,1
69676,722000,35-3023,3205050.0,1
...,...,...,...,...
49370,561700,15-1252,30.0,0
67472,712000,17-3022,30.0,0
49368,561700,15-1242,30.0,0
54787,611300,49-9051,30.0,0


+  **Non-standard List**. The clustering algorithm based on the list of non-standard NAICS and OCC Codes.

In [18]:
gmm_cluster(non_standard)

C:\Users\maurice\AppData\Local\Temp\ipykernel_31280\3600139752.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['TOT_EMP'] = df['TOT_EMP'].replace('**', np.nan)
C:\Users\maurice\AppData\Local\Temp\ipykernel_31280\3600139752.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TOT_EMP'] = pd.to_numeric(df['TOT_EMP'])
C:\Users\maurice\AppData\Local\Temp\ipykernel_31280\3600139752.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,NAICS,OCC_CODE,TOT_EMP,GMM_CLUSTER
74457,999001,25-2021,1254040.0,1
77532,999301,25-2021,1245480.0,1
74480,999001,25-9045,1033460.0,1
77555,999301,25-9045,1020950.0,1
74460,999001,25-2031,898000.0,1
...,...,...,...,...
77246,999300,51-5111,30.0,0
6953,99,41-9031,30.0,0
77887,999301,49-9092,30.0,0
74193,999000,53-4013,30.0,0


### Export Sheets to Excel

Export sheets to Excel.

In [ ]:
with pd.ExcelWriter('OEWS_NAICS_SOC.xlsx') as writer:
   standard_list.to_excel(writer, sheet_name='standard_list', index=False)
   non_standard.to_excel(writer, sheet_name='nonstandard_list', index=False)
   gmm_cluster(standard_list).to_excel(writer, sheet_name='standard_gmm_cluster', index=False)
   gmm_cluster(non_standard).to_excel(writer, sheet_name='nonstandard_gmm_cluster', index=False)